# devlog 2024-06-03

_author: Trevor Johnson_

ADRIOMakerCensus has been refactored to utilize the recently added GeoScope class heirarchy. This notebook tests the correct functionality of Census-based ADRIOs post-refactor by creating a DynamicGeo for each granularity and populating them with every attribute that is valid for their granularity.

Additional cases can be tested by changing the type of the scope objects, changing the year in time period or scope, or changing the includes attribute of the scope object.

In [1]:
from epymorph.data_shape import Shapes
from epymorph.data_type import CentroidDType
from epymorph.geo.adrio import adrio_maker_library
from epymorph.geo.dynamic import DynamicGeo
from epymorph.geo.spec import DynamicGeoSpec, Year
from epymorph.geography.us_census import DEFAULT_YEAR, StateScopeAll
from epymorph.simulation import AttributeDef

spec = DynamicGeoSpec(
    attributes=[
        AttributeDef('label', str, Shapes.N),
        AttributeDef('population', int, Shapes.N),
        # AttributeDef('population_by_age', int, Shapes.NxA(3)),
        # AttributeDef('population_by_age_x6', int, Shapes.NxA(6)),
        AttributeDef('centroid', CentroidDType, Shapes.N),
        AttributeDef('geoid', str, Shapes.N),
        AttributeDef('average_household_size', int, Shapes.N),
        AttributeDef('dissimilarity_index', float, Shapes.N),
        AttributeDef('commuters', int, Shapes.NxN),
        AttributeDef('gini_index', float, Shapes.N),
        AttributeDef('median_age', int, Shapes.N),
        AttributeDef('median_income', int, Shapes.N),
        AttributeDef('pop_density_km2', float, Shapes.N)
    ],
    time_period=Year(2020),
    scope=StateScopeAll(DEFAULT_YEAR),
    source={
        'label': 'Census:name',
        'population': 'Census',
        # 'population_by_age': 'Census',
        # 'population_by_age_x6': 'Census',
        'centroid': 'Census',
        'geoid': 'Census',
        'average_household_size': 'Census',
        'dissimilarity_index': 'Census',
        'commuters': 'Census',
        'gini_index': 'Census',
        'median_age': 'Census',
        'median_income': 'Census',
        'pop_density_km2': 'Census',
        'tract_median_income': 'Census'
    }
)

geo = DynamicGeo.from_library(spec, adrio_maker_library)

In [2]:
geo.validate()

In [3]:
from dataclasses import replace

from epymorph.geography.us_census import (BlockGroupScope, CountyScope,
                                          StateScope, TractScope)

spec = replace(spec, scope=StateScope.in_states(['04', '08']))
geo = DynamicGeo.from_library(spec, adrio_maker_library)

In [4]:
geo.validate()

In [5]:
spec = replace(spec, scope=CountyScope.in_counties(['35001', '04013', '04017']))
geo = DynamicGeo.from_library(spec, adrio_maker_library)

In [6]:
geo.validate()

In [7]:
spec = replace(spec, scope=TractScope.in_tracts(['35001000720', '35001000904', '35001000906',
                                                 '04027011405', '04027011407']), attributes=[
    AttributeDef('label', str, Shapes.N),
    AttributeDef('population', int, Shapes.N),
    # AttributeDef('population_by_age', int, Shapes.NxA(3)),
    # AttributeDef('population_by_age_x6', int, Shapes.NxA(6)),
    AttributeDef('centroid', CentroidDType, Shapes.N),
    AttributeDef('geoid', str, Shapes.N),
    AttributeDef('average_household_size', int, Shapes.N),
    AttributeDef('dissimilarity_index', float, Shapes.N),
    AttributeDef('gini_index', float, Shapes.N),
    AttributeDef('median_age', int, Shapes.N),
    AttributeDef('median_income', int, Shapes.N),
    AttributeDef('pop_density_km2', float, Shapes.N)
])

geo = DynamicGeo.from_library(spec, adrio_maker_library)

In [8]:
# tract and block group geos fetch shape file attributes prior to validating so that the kernel
# is not overwhelmed by several large shapefile requests in parallel
geo['centroid']
geo['pop_density_km2']
geo.validate()

In [9]:
spec = replace(spec, scope=BlockGroupScope.in_block_groups(['350010007201', '350010009041', '350010009061',
                                                            '040270114053', '040270114072']), attributes=[
    AttributeDef('label', str, Shapes.N),
    AttributeDef('population', int, Shapes.N),
    # AttributeDef('population_by_age', int, Shapes.NxA(3)),
    # AttributeDef('population_by_age_x6', int, Shapes.NxA(6)),
    AttributeDef('centroid', CentroidDType, Shapes.N),
    AttributeDef('geoid', str, Shapes.N),
    AttributeDef('average_household_size', int, Shapes.N),
    AttributeDef('gini_index', float, Shapes.N),
    AttributeDef('median_age', int, Shapes.N),
    AttributeDef('median_income', int, Shapes.N),
    AttributeDef('pop_density_km2', float, Shapes.N),
    AttributeDef('tract_median_income', int, Shapes.N)
])

geo = DynamicGeo.from_library(spec, adrio_maker_library)

In [10]:
geo['centroid']
geo['pop_density_km2']
geo.validate()

Gini Index cannot be retrieved for block group level, fetching tract level data instead.
